# Radial Basis Function Network (v 2.0)

## Section 0: Package initialisations and environment configuration

### Import relevant packages:

In [114]:
import numpy as np

# Interactive plotting
import plotly
import plotly.graph_objs as go
import plotly.offline as pyo

### Configure environment:

In [115]:
%config InlineBackend.figure_format = 'retina'
np.set_printoptions(precision=3)

# Activate Plotly Offline for Jupyter
pyo.init_notebook_mode(connected=True)

## Section 1: Loading data

In [116]:
"""
source.npz is a dictionary containing keys 'X' and 'Y', each of which holds an (N x P) matrix.
"""
# Load data
source = np.load("./Data/RBFN/source.npz")

# Create data and target variables
data = source['Y']
target = source['X']

print 'Shape of data: ', data.shape
print 'Shape of target: ', target.shape

Shape of data:  (20, 2)
Shape of target:  (20, 2)


## Section 2: Defining centres $\mathbf{\mu}$ and variances, $\mathbf{\sigma^2}$:

In [140]:
def calc_causality(data, target, scope=20, show_clusters=False):
    '''
    Calculate the causality index of data -> target.
    Inputs:
        data:               Input values (N x P) (suspected 'effect' variable)
        target:             Output values (N x P)(suspected 'cause' variable)
        scope:              Length of time series subset to perform causality calculations (int)
        show_clusters: If True, scatter plot of clusters will be shown for first set of calculation (Boolean)
    Returns:
        An array of normalised causality index (N - scope)
    '''
    
    def l2(A, axis=None):
        '''
        Calculates the L2-norm of a tensor, at a specified axis.
        Inputs:
            A:    A tensor.
            axis: Summation axis.
        Returns:
            L2-norm of tensor.
        '''
        return np.sqrt(np.sum(np.square(A), axis=1))
    
    def RMS(A):
        '''
        Perform a root-mean square operation.
        Input:
            A: A 1-D array (N)
        Returns:
            Root means square of A.
        '''
        return np.sqrt(np.mean(np.square(A)))
    
    def euclidean_dist(A, B=None):
        '''
        Calculate the euclidean distance for rows in matrix A and rows in matrix B.
        If B is None, calculates distances for rows between matrix A.
        Inputs:
            A: A matrix (a x P)
            B: A matrix (b x P)
        Returns:
            A distance matrix (a x b), indicating the distance of all non-i-th point to the i-th point. 
        ''' 
        # Define input matrices with expanded dimensions
        A_expanded = np.expand_dims(A, 2)

        # Calculate distance of each point and every other point
        if B is None:
            B_expanded = A_expanded
        else:
            B_expanded = np.expand_dims(B, 2)

        return l2(A_expanded - np.transpose(B_expanded, (2, 1, 0)), axis=1)

    def find_cluster_params(data):
        '''
        Given datapoints, find centres and variances for each radial basis.
        Assumptions:
            Each datapoint is itself a radial basis function.
            Variances are assumed to be identical for all basis functions. Calculated as four times the
                average L2 squared distance between all pairwise datapoints.
        Inputs:
            data: Data values (N x P)
        Returns:
            centres:  Centre coordinates for each radial basis (N x P)
            variance: Variance of radial basis (scalar)
        '''
        # Calculate Euclidean distance matrix (N x N)
        distances = euclidean_dist(data)

        # Obtain upper triangular of distances (excluding diagonals)
        distances[np.triu_indices(distances.shape[0], 0)] = 0

        # Calculate variance as four times of [average(Euclidean distance)]^2
        variance = 0.01 * (np.mean(distances) * np.true_divide(np.size(distances), np.sum(distances != 0)))**2

        return data, variance
    
    def MoG(data, k):
        '''
        Perform a Mixture of Gaussian clustering procedure.
        Assumptions:
            Clusters generated are dimensionally-independent (Naïve Bayes)
        Inputs:
            data: Data to cluster (N x P)
            k:    Number of clusters (N x P)
        Returns:
            Cluster centres (K x P)
            Cluster variances (K)
        '''
        def build_MoG(k):
            '''
            Build a TensorFlow MoG graph.
            Inputs:
                k: Number of clusters (N x P)
            Returns:
                Graph variables
            '''
    
    def visualise_clusters(data, centres, variances):
        '''
        Final result by colouring data points by clusters generated by Mixture of Gaussian algorithm
        Inputs:
            Centres:   Ccoordinates of cluster centres (K x P)
            Variances: Cluster variances (scalar)
        '''

        def _hex_to_rgb(colour_list):
            '''
            Convert hex values of type string to RGB of type int
            Input:
                colour_list: numpy array of type string (numColour x 1)
            Output:
                RGB: RGB component of type int (numColour x 3)
            '''
            RGB = np.array([])[np.newaxis,:].reshape(0,3)
            # Split hex values into R, G, B components
            # Convert components to int and store in RGB array
            for colour in colour_list:
                RGB = np.append(RGB, np.array([int(colour[1:3], 16), \
                                               int(colour[3:5], 16), \
                                               int(colour[5:7], 16)]).reshape(1, 3), axis=0)
            return RGB

        def _rgb_to_hex(RGB):
            '''
            Convert RGB of type int to hex string of format '#xxxxxx'
            Input:
                RGB: RGB component of type int (N x 3)
            Output:
                hex_colours: (N x 1)
            '''
            hex_colours = np.array([])
            # Convert RGB ints to a single hex string
            RGB = RGB.astype(int)
            for colour in RGB:
                hex_colours = np.append(hex_colours, '#{:02X}{:02X}{:02X}'.format(colour[0], colour[1], colour[2]))
            return hex_colours

        def get_colour_gradient(resp):
            '''
            Return the 'average' colour based on Plotly's default colour list and responsibility index
            Input:
                idx: responsibility index (N x K)
            Output:
                average_colour (N x 1)
            '''
            # Assert error if there are more colours than available colours
            N = resp.shape[0]
            K = resp.shape[1]
            try:
                assert K <= colour_list.shape
            except AssertionError:
                print 'Not enough colours to colour all K clusters. Consider increasing number of colours in colour_list.'

            # Matrix multiply resp (N x K) and RGB-ed colour_list (K x 3) to obtain 'average' colour
            # Multiply max resp to whiten less certain data points
            # assigned_colour = np.matmul(resp, _hex_to_rgb(colour_list[:K]))
            assigned_colour = np.matmul(np.eye(K, dtype='int')[np.argmax(resp, axis=1)], _hex_to_rgb(colour_list[:K]))
            white_layer = np.repeat(255, N * 3).reshape(N, 3)

            # Append white_layer to assigned_colour on axis=2
            # pre_whitened (N x K x 2)
            pre_whitened = np.append(assigned_colour[:,:,np.newaxis], white_layer[:,:,np.newaxis], axis=2)

            # Create weights (N x 2)
            # Second layer takes the converse of the maximum responsibility (N x 1)
            weights = np.append(np.ones(N)[:,np.newaxis], 1 - np.amax(resp, axis=1)[:, np.newaxis], axis=1)

            # Conform shape of weights to shape of pre_whitened
            weights = np.transpose(np.tile(weights, (3, 1, 1)), (1, 0, 2))

            # Perform weighted-average to colours
            whitened_colour = np.average(pre_whitened, weights=weights, axis=2)

            # Return matrix of colour in hex form
            return _rgb_to_hex(whitened_colour)

        def calc_ellipse_coordinates(centres, variances):
            '''
            Create x- and y-coordinates for ellipses for each cluster
            Assumptions:
                Dimension of data point is 2
            Returns:
                ellipse: x- and y-coordinates for K ellipses (N x K x D)
            '''
            # Create trace for region to encompass 95% of the points (using Chi-squared critical value)
            # Assuming joint independence and equal marginal variances

            # Chi-squared with df 2 and alpha=5%
            crit_val = 5.991

            # Obtain eigenvalues (K x D) and eigenvectors (K x D x D) for ellipses
            evalues, evectors = np.linalg.eig(variances)

            # Sort evalues and evectors in descending order of eigenvalue
            for k in range(len(evalues)):
                idx = np.argsort(evalues[k])[::-1]
                evalues[k] = evalues[k, idx]
                evectors[k] = evectors[k, :, idx]

            # Calculate axes length
            axis_lengths = np.sqrt(evalues * crit_val)

            # Calculate coordinates to trace ellipse
            t = np.arange(-np.pi, np.pi + np.pi / 50, np.pi / 50) # Parameter
            x = axis_lengths[:,0] * np.cos(t)[:, np.newaxis]
            y = axis_lengths[:,1] * np.sin(t)[:, np.newaxis]

            # Stack x- and y-coordinates along axis=2
            ellipse = np.stack([x, y], axis=2)

            # Define rotation angle (K x 1) and rotation matrix (K x 2 x 2)
            angles = np.arctan(evectors[:, 0, 1] / evectors[:, 0, 0])[:, np.newaxis]

            rot_mat = np.append(np.append(np.cos(angles), -np.sin(angles), axis=1)[:,:,np.newaxis], \
                                np.append(np.sin(angles), np.cos(angles), axis=1)[:,:,np.newaxis], axis=2)

            # Rotate ellipse
            ellipse = np.squeeze(np.matmul(np.expand_dims(ellipse, axis=2), np.expand_dims(rot_mat, axis=0)), axis=2)

            # Add centres to ellipse
            ellipse += centres[np.newaxis,:,:]

            return ellipse

        #######################
        ##  Function begins  ##
        #######################

        # Create ellipse coordinates
        ellipse = calc_ellipse_coordinates(centres, variances)

        # Define colour list as per Plotly's default colour list
        colour_list = np.array(['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b'])

        # Define blank figure
        figure = {
            'data': [],
            'layout': {}
        }
        
        # Create data trace
        data_trace = {
            'x': np.round(data[:, 0], 3),
            'y': np.round(data[:, 1], 3),
            'mode': 'markers',
            'hoverinfo': 'none',
            'marker': {
                'color': colour_list[-1]
            }
        }
        
        figure['data'].append(data_trace)

        for k in range(len(centres)):
            # Create trace for cluster centres
            centre_trace = {
                'x': np.round([centres[k][0]], 3),
                'y': np.round([centres[k][1]], 3),
                'name': 'Cluster {}'.format(k + 1),
                'mode': 'markers',
                'marker': {
                        'size': 12,
                        'symbol': 'diamond',
                        'color': colour_list[k],
                        'line': {'width': 3}
                    }   
            }

            # Create trace for region encompassing 95% of data points
            variance_trace = {
                'x': ellipse[:,k,:][:,0],
                'y': ellipse[:,k,:][:,1],
                'hoverinfo': 'none',
                'mode': 'lines',
                'name': 'Cluster {}'.format(k + 1),
                'marker': {
                    'color': colour_list[k]
                }
            }

            # Add cluster trace
            for trace in [centre_trace, variance_trace]:
                figure['data'].append(trace)

        # Generate figure layout
        figure['layout'] = go.Layout(
            width = 900,
            height = 900,
            showlegend = False,
            title = 'Clusters Visualisation',
            xaxis = {'range': [-4, 4], 'autorange': True},
            yaxis = {'range': [-5, 2], 'autorange': True}
        )

        return pyo.iplot(figure)

    def calc_radial_basis_activations(data, centres, variances):
        '''
        Calculates the activations for the hidden radial basis layer.
        Inputs:
            data: Data values (N x P)
            centres: RBF centres (K x P)
            variances: RBF variances (scalar)
        Returns:
            Radial basis activations (N x K)
        '''
        # Calculate Gaussian exponent
        actv = np.exp( - np.divide(euclidean_dist(data, centres)**2, 2 * np.transpose(variances)))
            
        return actv

    def train_RBFN_weights(data, target, centres, variances):
        '''
        Train the weights of a Radial Basis Function Network by solving for values in parameter \mathbf{\alpha}.
        Normalise the activations before solving for \mathbf{\alpha}, 
            and re-normalising the values of \mathbf{\alpha} after.

        Inputs:
            data:   Data values (N x P)
            target: Target values (N x P)
            centres: Cluster centres (K x P)
            variances: Cluster variances (K)
        Returns:
            Trained weights, \mathbf{\alpha} (K x P)
        '''
        # Obtain activations
        activations = calc_radial_basis_activations(data, centres, variances)
        
        # Store the L2-norms of columns of activations in a matrix
        L2 = l2(activations, axis=0)

        # Normalise activation values by their L2-norms
        #actvn_norm = np.divide(activations, np.expand_dims(L2, axis=1))
        actvn_norm = activations
        
        # Solve system of linear equations for alpha
        # weights = np.linalg.solve(actvn_norm, target)
        weights = np.matmul(np.linalg.inv(np.matmul(actvn_norm.T, actvn_norm)), np.matmul(actvn_norm.T, target))

        # Return re-normalised alpha
        return weights
        # return np.divide(weights, np.expand_dims(L2, axis=1))

    def RBFN_calc(data, centres, variances, weights):
        '''
        Calculate the output of the trained RBFN.
        Inputs:
            data:      Data values (N x P)
            centres:   RBF centres (K x P)
            variances: RBF variances (scalar)
            weights:   Trained weights (K x P)
        Returns:
            Predicted target value (N x P), assuming data and target have same dimensionality
        '''
        # Calculate radial basis activations
        actv = calc_radial_basis_activations(data, centres, variances)
        
        # Calculate and return predicted output
#         return np.matmul(actv, weights)
        return np.matmul(np.expand_dims(actv, 0), weights)
    
    
    ###################
    # Function Begins #
    ###################
    
    # Error checking
    try:
        assert data.shape[0] >= scope
    except:
        print 'Error: Time series is shorter than scope. Please ensure scope is not shorter than the length of your time series.'
    
    # Define variables
    N = data.shape[0]
    causality = np.zeros(N - scope + 1)
    
    # TODO: Use compressive sensing algorithm
    pred = np.zeros((scope, 2))
    
    for i in range(N - scope + 1):
        print 'Calculating causality for time index: {}'.format(i + 1)
        # Define subset of working data and target
        scoped_data = data[i:(scope + i),:]
        scoped_target = target[i:(scope + i),:]
        
        # Define error variable
        error = np.zeros(scope)
        
        # Visualise clusters to see if centres and variances are appropriate
        if (i == 0) and (show_clusters == True):
#             centres, variances = find_cluster_params(data=scoped_data) 
            %store -r result
            centres = result[0]['cluster_centres'][-1]
            variances = result[0]['cluster_variances'][-1]
            
            visualise_clusters(data, centres, variances)
        
#         for j in range(1):
        for j in range(scope):
            # Obtain centres and variances of RBFs using leave-one-out scoped dataset
#             centres, variances = find_cluster_params(data=np.delete(scoped_data, j, 0))
            
            %store -r result
            centres = result[0]['cluster_centres'][-1]
            variances = result[0]['cluster_variances'][-1]
            
            # Train weights of RBFN using leave-one-out scoped dataset
            weights = train_RBFN_weights(
                data = np.delete(scoped_data, j, 0), 
                target = np.delete(scoped_target, j, 0),
                centres = centres,
                variances = variances
            )
            
            # Calculate predicted value of output
            prediction = RBFN_calc(
                data = scoped_data[j,:][np.newaxis, :],
                centres = centres,
                variances = variances,
                weights = weights
            )
            
#             print prediction
#             print scoped_target[j, :]
#             print
            
            # Calculate error
            error[j] = l2(prediction - scoped_target[j,:])
            pred[j,:] = prediction
        
        print error
        
        # Calculate \delta
        delta = RMS(error) / RMS(l2(scoped_data - np.mean(scoped_data, axis=0)))
        
        print delta
        
        # Calculate causality index
        causality[i] = np.exp( - delta / 5.)
    
    trace = go.Scatter(
        x = target[:,-1],
        y = pred[:,-1],
        mode = 'markers',
    )
    
    pyo.iplot(go.Figure(data=go.Data([trace]), layout=go.Layout()))
        
    return causality

In [142]:
calc_causality(data=data, target=target, scope=20, show_clusters=True)

Calculating causality for time index: 1


[ 1.002  0.437  0.56   0.627  0.407  0.118  0.941  0.249  0.202  0.258
  0.678  0.393  0.181  0.382  0.289  0.17   0.774  0.07   0.923  0.251]
1.82568136168


array([ 0.694])

In [141]:
calc_causality(data=target, target=data, scope=20, show_clusters=True)

Calculating causality for time index: 1


[ 1.004  0.381  0.559  0.752  0.561  0.181  0.218  0.856  0.328  0.469
  0.684  0.394  0.349  0.628  0.525  0.104  0.107  0.758  0.326  0.273]
2.09853854665


array([ 0.657])

In [70]:
def l2(A, axis=None):
    '''
    Calculates the L2-norm of a tensor, at a specified axis.
    Inputs:
        A:    A tensor.
        axis: Summation axis.
    Returns:
        L2-norm of tensor.
    '''
    return np.sqrt(np.sum(np.square(A), axis=1))

def RMS(A):
    '''
    Perform a root-mean square operation.
    Input:
        A: A 1-D array (N)
    Returns:
        Root means square of A.
    '''
    return np.sqrt(np.mean(np.square(A)))

def euclidean_dist(A, B=None):
    '''
    Calculate the euclidean distance for rows in matrix A and rows in matrix B.
    If B is None, calculates distances for rows between matrix A.
    Inputs:
        A: A matrix (a x P)
        B: A matrix (b x P)
    Returns:
        A distance matrix (a x b), indicating the distance of all non-i-th point to the i-th point. 
    ''' 
    # Define input matrices with expanded dimensions
    A_expanded = np.expand_dims(A, 2)

    # Calculate distance of each point and every other point
    if B is None:
        B_expanded = A_expanded
    else:
        B_expanded = np.expand_dims(B, 2)

    return l2(A_expanded - np.transpose(B_expanded, (2, 1, 0)), axis=1)

def find_cluster_params(data):
    '''
    Given datapoints, find centres and variances for each radial basis.
    Assumptions:
        Each datapoint is itself a radial basis function.
        Variances are assumed to be identical for all basis functions. Calculated as four times the
            average L2 squared distance between all pairwise datapoints.
    Inputs:
        data: Data values (N x P)
    Returns:
        centres:  Centre coordinates for each radial basis (N x P)
        variance: Variance of radial basis (scalar)
    '''
    # Calculate Euclidean distance matrix (N x N)
    distances = euclidean_dist(data)

    # Obtain upper triangular of distances (excluding diagonals)
    distances[np.triu_indices(distances.shape[0], 0)] = 0

    # Calculate variance as four times of [average(Euclidean distance)]^2
    variance = 0.01 * (np.mean(distances) * np.true_divide(np.size(distances), np.sum(distances != 0)))**2

    return data, variance

def MoG(data, k):
    '''
    Perform a Mixture of Gaussian clustering procedure.
    Assumptions:
        Clusters generated are dimensionally-independent (Naïve Bayes)
    Inputs:
        data: Data to cluster (N x P)
        k:    Number of clusters (N x P)
    Returns:
        Cluster centres (K x P)
        Cluster variances (K)
    '''
    def build_MoG(k):
        '''
        Build a TensorFlow MoG graph.
        Inputs:
            k: Number of clusters (N x P)
        Returns:
            Graph variables
        '''

def visualise_clusters(data, centres, variances):
    '''
    Final result by colouring data points by clusters generated by Mixture of Gaussian algorithm
    Inputs:
        Centres:   Ccoordinates of cluster centres (K x P)
        Variances: Cluster variances (scalar)
    '''

    def _hex_to_rgb(colour_list):
        '''
        Convert hex values of type string to RGB of type int
        Input:
            colour_list: numpy array of type string (numColour x 1)
        Output:
            RGB: RGB component of type int (numColour x 3)
        '''
        RGB = np.array([])[np.newaxis,:].reshape(0,3)
        # Split hex values into R, G, B components
        # Convert components to int and store in RGB array
        for colour in colour_list:
            RGB = np.append(RGB, np.array([int(colour[1:3], 16), \
                                           int(colour[3:5], 16), \
                                           int(colour[5:7], 16)]).reshape(1, 3), axis=0)
        return RGB

    def _rgb_to_hex(RGB):
        '''
        Convert RGB of type int to hex string of format '#xxxxxx'
        Input:
            RGB: RGB component of type int (N x 3)
        Output:
            hex_colours: (N x 1)
        '''
        hex_colours = np.array([])
        # Convert RGB ints to a single hex string
        RGB = RGB.astype(int)
        for colour in RGB:
            hex_colours = np.append(hex_colours, '#{:02X}{:02X}{:02X}'.format(colour[0], colour[1], colour[2]))
        return hex_colours

    def get_colour_gradient(resp):
        '''
        Return the 'average' colour based on Plotly's default colour list and responsibility index
        Input:
            idx: responsibility index (N x K)
        Output:
            average_colour (N x 1)
        '''
        # Assert error if there are more colours than available colours
        N = resp.shape[0]
        K = resp.shape[1]
        try:
            assert K <= colour_list.shape
        except AssertionError:
            print 'Not enough colours to colour all K clusters. Consider increasing number of colours in colour_list.'

        # Matrix multiply resp (N x K) and RGB-ed colour_list (K x 3) to obtain 'average' colour
        # Multiply max resp to whiten less certain data points
        # assigned_colour = np.matmul(resp, _hex_to_rgb(colour_list[:K]))
        assigned_colour = np.matmul(np.eye(K, dtype='int')[np.argmax(resp, axis=1)], _hex_to_rgb(colour_list[:K]))
        white_layer = np.repeat(255, N * 3).reshape(N, 3)

        # Append white_layer to assigned_colour on axis=2
        # pre_whitened (N x K x 2)
        pre_whitened = np.append(assigned_colour[:,:,np.newaxis], white_layer[:,:,np.newaxis], axis=2)

        # Create weights (N x 2)
        # Second layer takes the converse of the maximum responsibility (N x 1)
        weights = np.append(np.ones(N)[:,np.newaxis], 1 - np.amax(resp, axis=1)[:, np.newaxis], axis=1)

        # Conform shape of weights to shape of pre_whitened
        weights = np.transpose(np.tile(weights, (3, 1, 1)), (1, 0, 2))

        # Perform weighted-average to colours
        whitened_colour = np.average(pre_whitened, weights=weights, axis=2)

        # Return matrix of colour in hex form
        return _rgb_to_hex(whitened_colour)

    def calc_ellipse_coordinates(centres, variances):
        '''
        Create x- and y-coordinates for ellipses for each cluster
        Assumptions:
            Dimension of data point is 2
        Returns:
            ellipse: x- and y-coordinates for K ellipses (N x K x D)
        '''
        # Create trace for region to encompass 95% of the points (using Chi-squared critical value)
        # Assuming joint independence and equal marginal variances

        # Chi-squared with df 2 and alpha=5%
        crit_val = 5.991 

        # Calculate axes length
        axis_lengths = np.sqrt(variances * crit_val)

        # Calculate coordinates to trace ellipse
        t = np.arange(-np.pi, np.pi + np.pi / 50, np.pi / 50) # Parameter
        x = np.transpose(centres[:,0][:, np.newaxis]) + axis_lengths * np.cos(t)[:, np.newaxis]
        y = np.transpose(centres[:,1][:, np.newaxis]) + axis_lengths * np.sin(t)[:, np.newaxis]

        # Stack x- and y-coordinates along axis=2
        ellipse = np.stack([x, y], axis=2)

        return ellipse

    #######################
    ##  Function begins  ##
    #######################

    # Create ellipse coordinates
    ellipse = calc_ellipse_coordinates(centres, variances)

    # Define colour list as per Plotly's default colour list
    colour_list = np.array(['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b'])

    # Define blank figure
    figure = {
        'data': [],
        'layout': {}
    }

    # Create data trace
    data_trace = {
        'x': np.round(data[:, 0], 3),
        'y': np.round(data[:, 1], 3),
        'mode': 'markers',
        'hoverinfo': 'none',
        'marker': {
            'color': colour_list[-1]
        }
    }

    figure['data'].append(data_trace)

    for k in range(len(centres)):
        # Create trace for cluster centres
        centre_trace = {
            'x': np.round([centres[k][0]], 3),
            'y': np.round([centres[k][1]], 3),
            'name': 'Cluster {}'.format(k + 1),
            'mode': 'markers',
            'marker': {
                    'size': 12,
                    'symbol': 'diamond',
                    'color': colour_list[k],
                    'line': {'width': 3}
                }   
        }

        # Create trace for region encompassing 95% of data points
        variance_trace = {
            'x': ellipse[:,k,:][:,0],
            'y': ellipse[:,k,:][:,1],
            'hoverinfo': 'none',
            'mode': 'lines',
            'name': 'Cluster {}'.format(k + 1),
            'marker': {
                'color': colour_list[k]
            }
        }

        # Add cluster trace
        for trace in [centre_trace, variance_trace]:
            figure['data'].append(trace)

    # Generate figure layout
    figure['layout'] = go.Layout(
        width = 900,
        height = 900,
        showlegend = False,
        title = 'Clusters Visualisation',
        xaxis = {'range': [-4, 4], 'autorange': True},
        yaxis = {'range': [-5, 2], 'autorange': True}
    )

    return pyo.iplot(figure)

def calc_radial_basis_activations(data, centres, variances):
    '''
    Calculates the activations for the hidden radial basis layer.
    Inputs:
        data: Data values (N x P)
        centres: RBF centres (K x P)
        variances: RBF variances (scalar)
    Returns:
        Radial basis activations (N x K)
    '''
    # Calculate Gaussian exponent
    actv = np.exp( - np.divide(euclidean_dist(data, centres)**2, 2 * np.transpose(variances)))

    return actv

def train_RBFN_weights(data, target, centres, variances):
    '''
    Train the weights of a Radial Basis Function Network by solving for values in parameter \mathbf{\alpha}.
    Normalise the activations before solving for \mathbf{\alpha}, 
        and re-normalising the values of \mathbf{\alpha} after.

    Inputs:
        data:   Data values (N x P)
        target: Target values (N x P)
        centres: Cluster centres (K x P)
        variances: Cluster variances (K)
    Returns:
        Trained weights, \mathbf{\alpha} (K x P)
    '''
    # Obtain activations
    activations = calc_radial_basis_activations(data, centres, variances)
    
    print activations.shape

    # Store the L2-norms of columns of activations in a matrix
    L2 = l2(activations, axis=0)

    # Normalise activation values by their L2-norms
    actvn_norm = activations
#     actvn_norm = np.divide(activations, np.expand_dims(L2, axis=1))
    
#     return actvn_norm

    # Solve system of linear equations for alpha
    weights = np.matmul(np.linalg.inv(np.matmul(actvn_norm.T, actv_norm)), np.matmul(actv_norm.T, target))

    # Return re-normalised alpha
#     return np.divide(weights, np.expand_dims(L2, axis=1))
    return weights

def RBFN_calc(data, centres, variances, weights):
    '''
    Calculate the output of the trained RBFN.
    Inputs:
        data:      Data values (N x P)
        centres:   RBF centres (K x P)
        variances: RBF variances (scalar)
        weights:   Trained weights (K x P)
    Returns:
        Predicted target value (N x P), assuming data and target have same dimensionality
    '''
    # Calculate radial basis activations
    actv = calc_radial_basis_activations(data, centres, variances)

    # Calculate and return predicted output
    return np.matmul(actv, weights)


In [71]:
# test_data = np.random.randn(5,3)
# test_target = np.eye(5, 3)

%store -r result
# centres, variances = find_cluster_params(np.eye(2))
centres = result[0]['cluster_centres'][-1]
variances = result[0]['cluster_variances'][-1]
print variances
visualise_clusters(data, centres, variances)
actv_norm = train_RBFN_weights(data, target, centres, variances)

[ 0.009  0.016]


(20, 2)


In [46]:
print np.linalg.inv(actv_norm) 

LinAlgError: Last 2 dimensions of the array must be square

In [48]:
import cvxpy as cvx

In [49]:
target.flatten()

array([ 0.649,  0.843,  0.843,  0.49 ,  0.49 ,  0.925,  0.925,  0.258,
        0.258,  0.708,  0.708,  0.765,  0.765,  0.665,  0.665,  0.824,
        0.824,  0.537,  0.537,  0.92 ,  0.92 ,  0.272,  0.272,  0.733,
        0.733,  0.723,  0.723,  0.74 ,  0.74 ,  0.711,  0.711,  0.76 ,
        0.76 ,  0.676,  0.676,  0.811,  0.811,  0.567,  0.567,  0.908])

In [56]:
A = actv_norm
b = target
# result = np.linalg.lstsq(A,b)[0]

# print result

vx = cvx.Variable(A.shape[1], b.shape[1])
objective = cvx.Minimize(cvx.norm(vx, 1))
constraints = [A*vx == b]
prob = cvx.Problem(objective, constraints)
result = prob.solve(verbose=True)


ECOS 2.0.4 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  +0.000e+00  -0.000e+00  +7e+00  6e-01  1e-02  1e+00  9e-01    ---    ---    2  1  - |  -  - 
 1  +1.090e+02  +1.252e+02  +4e-01  5e-01  5e-03  2e+01  5e-02  0.9661  2e-02   1  1  1 |  0  0
 2  +1.060e+04  +1.215e+04  +4e-03  5e-01  5e-03  2e+03  6e-04  0.9890  1e-04   2  1  1 |  0  0
 3  +9.564e+05  +1.096e+06  +5e-05  5e-01  5e-03  1e+05  6e-06  0.9890  1e-04   2  1  1 |  0  0
 4  +8.625e+07  +9.888e+07  +5e-07  5e-01  5e-03  1e+07  7e-08  0.9890  1e-04   2  0  0 |  0  0
 5  +7.779e+09  +8.918e+09  +6e-09  5e-01  5e-03  1e+09  8e-10  0.9890  1e-04   2  0  0 |  0  0

PRIMAL INFEASIBLE (within feastol=2.6e-10).
Runtime: 0.000782 seconds.



In [341]:
vx.value

In [64]:
np.matmul(np.linalg.inv(np.matmul(A.T, A)), np.matmul(A.T, b))

array([[ 0.64 ,  0.702],
       [ 0.68 ,  0.653]])

In [313]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy.optimize as spopt
import scipy.fftpack as spfft
import scipy.ndimage as spimg
import cvxpy as cvx

In [314]:
# sum of two sinusoids
n = 5000
t = np.linspace(0, 1/8, n)
y = np.sin(1394 * np.pi * t) + np.sin(3266 * np.pi * t)
yt = spfft.dct(y, norm='ortho')

# extract small sample of signal
m = 500 # 10% sample
ri = np.random.choice(n, m, replace=False) # random sample of indices
ri.sort() # sorting not strictly necessary, but convenient for plotting
t2 = t[ri]
y2 = y[ri]

In [315]:
# create idct matrix operator
A = spfft.idct(np.identity(n), norm='ortho', axis=0)
A = A[ri]

# do L1 optimization
vx = cvx.Variable(n)
objective = cvx.Minimize(cvx.norm(vx, 1))
constraints = [A*vx == y2]
prob = cvx.Problem(objective, constraints)
result = prob.solve(verbose=True)


ECOS 2.0.4 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  +0.000e+00  -0.000e+00  +5e+03  1e+00  1e-02  1e+00  5e-01    ---    ---    0  1  - |  -  - 
 1  -3.421e-04  -0.000e+00  +6e+01  5e-01  1e-04  1e-02  6e-03  0.9890  1e-04   0  0  0 |  0  0
 2  -3.222e-06  -0.000e+00  +6e-01  1e-02  1e-06  1e-04  6e-05  0.9890  1e-04   0  0  0 |  0  0
 3  -2.945e-08  -0.000e+00  +7e-03  1e-04  2e-08  1e-06  7e-07  0.9890  1e-04   0  0  0 |  0  0
 4  -2.576e-10  -0.000e+00  +8e-05  1e-06  2e-10  2e-08  8e-09  0.9890  1e-04   0  0  0 |  0  0
 5  -2.098e-12  -0.000e+00  +9e-07  2e-08  2e-12  2e-10  9e-11  0.9890  1e-04   0  0  0 |  0  0
 6  -1.493e-14  -0.000e+00  +9e-09  2e-10  2e-14  2e-12  9e-13  0.9890  1e-04   0  0  0 |  0  0

OPTIMAL (within feastol=1.8e-10, reltol=6.4e+05, abstol=9.5e-09).
Runtime: 24.711984 seconds.



In [317]:
vx.value

matrix([[ -2.93137221e-26],
        [ -8.77209156e-28],
        [  4.15287077e-26],
        ..., 
        [  2.07341027e-26],
        [  6.87202509e-26],
        [  6.95054716e-26]])